# Instructions

Ce notebook ne doit contenir que votre script servant à l'entrainement de votre modèle. Nous devons pouvoir l'exécuter en cliquant sur *Exécution -> Tout exécuter*.

Veuillez également ajouter des commentaires dans votre code pour expliquer ce que vous faites. N'hésitez pas à ajouter des blocs de textes (cliquez sur le bouton *+ Texte* en dessous du menu) pour ajouter plus d'explications.

Vous devrez déposer sur Moodle une archive au format .zip contenant un dossier avec vos noms.

Dans ce dossier, nous devons retrouver les deux notebooks (training et testing) ainsi qu'un nouveau dossier *models* contenant les poids de vos modèles entrainés, et si nécessaire un dossier *datasets* contenant d'autres données utilisée pour effectuer l'apprentissage de vos modèles (données obtenues par récupération sur le web "web scraping"  ou bien augmentation de données "data augmentation"). Si vous effectuez de l'augmentation de données, fournissez aussi le code pour la réaliser dans le notebook.

In [11]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer, DataCollatorWithPadding
import evaluate

np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', precision=3, suppress=False, threshold=1000, formatter=None)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

False
0


In [12]:
dataset = Dataset.from_pandas(pd.read_csv('fake_train.csv'))

ds_train, ds_test = dataset.train_test_split(test_size=0.2).values()

In [13]:
id2label = {0: "News", 1: "Fake News"}
label2id = {"News": 0, "Fake News": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2,
    id2label=id2label,
    label2id=label2id
).cuda()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def preprocess_function(examples):
    return tokenizer(examples['data'], truncation=True, padding=True, max_length=512)

tokenized_train = ds_train.map(preprocess_function, batched=True)
tokenized_test = ds_test.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="defi_3_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    push_to_hub=False,
)

# pour save mais bug sur mon pc
#    save_strategy="epoch",
#    load_best_model_at_end=True,

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.save_model("saved_model")
tokenizer.save_pretrained("saved_model")